<a href="https://colab.research.google.com/github/deacs11/CrewAI_Grant_Funding_Opportunity_Finder_-_Eligibility_Checker_Crew/blob/main/CrewAI_Grant_Funding_Opportunity_Finder_%26_Eligibility_Checker_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Install necessary libraries
# Installs crewai, tools, OpenAI client, Colab module, and search tool
# Might need pypdf later if handling PDFs: !pip install pypdf
!pip install crewai crewai-tools langchain-openai google-colab duckduckgo-search==5.3.1b1 -q

print("Library installation completed!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.1 MB/s et

In [3]:
# @title 2. Import modules and configure API keys from secrets
import os
from google.colab import userdata # To read Colab secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool # Base tools
# from crewai_tools import PDFSearchTool # Consider adding if grants are often PDFs
from langchain_openai import ChatOpenAI
import json

# --- API key configuration from Colab secrets ---
print("Attempting to load API keys from Secrets...")
try:
    # Reads the OpenAI key from the secret 'OPENAI_API_KEY'
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    # Reads the Serper key from the secret 'SERPER_API_KEY'
    os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
    print("-> API key environment variables set successfully.")
except Exception as e:
    print(f"!!! Error loading API keys from secrets: {e}")
    print("!!! Make sure you have set them correctly in the Colab Secrets panel")
    print("!!! and enabled 'Notebook access' for both.")

# Debug block to verify keys
print("-" * 20)
retrieved_key = os.environ.get("OPENAI_API_KEY")
if retrieved_key:
    print(f"OpenAI key FOUND in environment: '{retrieved_key[:5]}...{retrieved_key[-4:]}'")
else:
    print("!!! OpenAI key NOT FOUND.")
print("-" * 20)
retrieved_serper_key = os.environ.get("SERPER_API_KEY")
if retrieved_serper_key:
     print(f"Serper key FOUND in environment: '{retrieved_serper_key[:5]}...{retrieved_serper_key[-4:]}'")
else:
    print("--- Serper key NOT FOUND (required for OpportunitySearcher).")
print("-" * 20)

Attempting to load API keys from Secrets...
-> API key environment variables set successfully.
--------------------
OpenAI key FOUND in environment: 'sk-pr...DKkA'
--------------------
Serper key FOUND in environment: '7354c...dc56'
--------------------


In [6]:
# @title 3. Define project and organization URLs & IMPORTANT disclaimer

# --- PROVIDE URLs CONTAINING THE PROJECT AND ORGANIZATION DETAILS ---

# Example using Doctors Without Borders (MSF) work in Yemen

# URL for the project description (Describes their activities and focus in Yemen)
project_info_url = "https://www.doctorswithoutborders.org/what-we-do/countries/yemen" # VERIFIED WORKING URL (MSF USA)
# Alt: https://www.msf.org/yemen (MSF International - might have slightly different details)

# URL for the organization details (Their main 'About Us' page)
organization_info_url = "https://www.doctorswithoutborders.org/about-us" # VERIFIED WORKING URL (MSF USA)

# Optional: Add specific keywords if not easily extracted from the project page
# Adjust keywords for the new sector and project focus
project_keywords_override = "humanitarian aid, medical assistance, Yemen, conflict zone, emergency medicine, malnutrition, healthcare access, non-profit, NGO, international aid" # UPDATED KEYWORDS
# --------------------------------------------------------------------------

# --- !!! IMPORTANT DISCLAIMER !!! ---
print("*"*70)
print("!!! DISCLAIMER: Grant Opportunity Finder & Eligibility Assistant !!!")
print("This tool performs automated searches and preliminary eligibility checks.")
print("It is for informational purposes ONLY and may miss opportunities or")
print("misinterpret complex eligibility criteria or website content.")
print("Eligibility requirements MUST be verified directly with the official")
print("grant documentation and funding agency.")
print("This tool DOES NOT guarantee funding or eligibility and provides NO")
print("financial or legal advice.")
print("*"*70)
print("\nProject and Organization URLs Set for Analysis.")
print(f"Project URL: {project_info_url}")
print(f"Organization URL: {organization_info_url}")

**********************************************************************
!!! DISCLAIMER: Grant Opportunity Finder & Eligibility Assistant !!!
This tool performs automated searches and preliminary eligibility checks.
It is for informational purposes ONLY and may miss opportunities or
misinterpret complex eligibility criteria or website content.
Eligibility requirements MUST be verified directly with the official
grant documentation and funding agency.
This tool DOES NOT guarantee funding or eligibility and provides NO
financial or legal advice.
**********************************************************************

Project and Organization URLs Set for Analysis.
Project URL: https://www.doctorswithoutborders.org/what-we-do/countries/yemen
Organization URL: https://www.doctorswithoutborders.org/about-us


In [7]:
# @title 4. Select LLM and initialize tools

# --- Choose the language model (LLM) ---
try:
    llm = ChatOpenAI(model="gpt-4-turbo")
    print(f"LLM ({llm.model_name}) initialized successfully.")
except Exception as e:
    print(f"!!! Error initializing ChatOpenAI: {e}. Verify API key.")
    raise

# --- Initialize tools ---
# Web reading tool is now ESSENTIAL
search_tool = None
web_reader_tool = None
try:
    search_tool = SerperDevTool() # Still needed for grant searching
    print("SerperDevTool initialized.")
    web_reader_tool = WebsiteSearchTool() # Crucial for reading input URLs
    print("WebsiteSearchTool initialized.")
except Exception as e:
    print(f"--- Warning: Error initializing tools: {e}. Core functionality might fail.")

# Assemble the list of tools that were successfully initialized
available_tools = [tool for tool in [search_tool, web_reader_tool] if tool is not None]
if not web_reader_tool:
     print("!!! CRITICAL WARNING: Web reading tool failed to initialize. The crew cannot process URLs.")
     # Optionally raise an error here if web reading is mandatory
     # raise ValueError("Web reading tool is essential and failed to initialize.")
if not search_tool:
    print("--- Warning: Search tool failed to initialize. Grant searching will be impaired.")

print(f"Tools actually available: {[t.name for t in available_tools] if available_tools else 'None'}")

LLM (gpt-4-turbo) initialized successfully.
SerperDevTool initialized.
WebsiteSearchTool initialized.
Tools actually available: ['Search the internet with Serper', 'Search in a specific website']


/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


In [12]:
# @title 5. Define agents for grant finding crew

print("Defining grant finding agents (URL Input Version)...")

if 'llm' not in locals() or llm is None:
     raise ValueError("LLM not initialized.")
if not available_tools or not web_reader_tool: # Check specifically for web_reader
     raise ValueError("Web reading tool is essential for URL input and is not available.")

# --- Agent 0a: Project Info Extractor ---
project_info_extractor = Agent(
    role='Web Content Analyst for Projects',
    goal=f"Visit the provided project URL ('{project_info_url}') and extract key information: a concise summary of the project's goals and activities, target beneficiaries/location, and relevant keywords.",
    backstory=(
        "You specialize in reading web pages (like 'About' or 'Projects' sections) and extracting the essential details about a specific initiative. "
        "You identify the core purpose, activities, and scope relevant for understanding the project for funding purposes."
    ),
    tools=[web_reader_tool], # Explicitly needs web reading
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'project_info_extractor' (NEW) defined.")

# --- Agent 0b: Organization Info Extractor ---
org_info_extractor = Agent(
    role='Web Content Analyst for Organizations',
    goal=f"Visit the provided organization URL ('{organization_info_url}') and extract key details relevant for grant eligibility: organization type (e.g., non-profit, NGO, academic), primary location/base, mission/focus areas, and legal status (e.g., 501(c)(3)) if mentioned.",
    backstory=(
        "You specialize in reading organizational websites ('About Us', 'Mission', 'Legal Status' pages) to extract factual details pertinent to grant applications. "
        "You focus on identifying the organization's nature, scope, and status."
    ),
    tools=[web_reader_tool], # Explicitly needs web reading
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'org_info_extractor' (NEW) defined.")


# --- Agent 1: Opportunity Searcher ---
opportunity_searcher = Agent(
    role='Grant Prospecting Specialist',
    # Goal now uses extracted info, and optionally the keyword override
    goal=(f"Identify 5-7 potentially relevant grant funding opportunities based on the extracted project details (provided in context) "
          f"and these additional keywords (if provided): '{project_keywords_override}'. Focus on relevant funding areas (e.g., international development, environment, health) "
          f"and geographical scope (e.g., Southeast Asia, based on extracted project info). Provide URLs to specific grant pages."),
    backstory=(
        "You are an expert in navigating the grant funding landscape using targeted web searches based on project specifics. "
        "You find opportunities aligned with goals, location, and keywords."
    ),
    tools=[search_tool] if search_tool else [],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=7
)
print("- Agent 'opportunity_searcher' (Modified) defined.")

# --- Agent 2: Grant Info Extractor ---
grant_info_extractor = Agent(
    role='Grant Information Analyst',
    goal=("For each potential grant opportunity URL/lead provided, visit the source and extract the following key information accurately:\n"
          "- Grant Name / Program Title\n"
          "- Funding Agency / Organization\n"
          "- Application Deadline (if specified)\n"
          "- Funding Range or Max Amount (if specified)\n"
          "- Brief Summary of Grant Purpose/Goal\n"
          "- **Exact Eligibility Criteria Text:** (Copy the section describing who can apply)"),
    backstory=(
        "You are meticulous at reading grant guidelines and extracting critical data points. You navigate websites or documents (using tools) "
        "to find specific sections detailing deadlines, funding levels, purpose, and especially the explicit eligibility requirements. Accuracy is paramount."
    ),
    tools=available_tools, # Needs web reading, potentially PDF reading
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=10
)
print("- Agent 'grant_info_extractor' defined.")

# --- Agent 3: Eligibility Matcher ---
eligibility_matcher = Agent(
    role='Preliminary Eligibility Assessor',
    # Goal now uses extracted org details
    goal=("Compare the extracted 'Exact Eligibility Criteria Text' for each grant opportunity against the user's extracted organization details (provided in context). "
          "Perform a *preliminary* check and categorize eligibility as: 'Potentially Eligible', 'Likely Ineligible' (state reason), or 'Unclear / Needs Verification' (state reason). "
          "**Emphasize this is NOT a definitive assessment.**"),
    backstory=(
        "You carefully compare stated requirements against a profile based *only* on the extracted text. "
        "You flag clear mismatches or ambiguities, always recommending further human verification."
    ),
    tools=[], # Performs comparison based on context
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=5
)
print("- Agent 'eligibility_matcher' (Modified) defined.")

# --- Agent 4: Report Compiler ---
report_compiler = Agent(
    role='Funding Opportunity Report Generator',
    goal='Compile a structured report summarizing the potentially relevant grant opportunities found. For each opportunity assessed as "Potentially Eligible" or "Unclear", include: Grant Name, Funding Agency, Deadline, Funding Range, Purpose Summary, the Extracted Eligibility Criteria text, and the Preliminary Eligibility Assessment result (with reasoning for ineligible/unclear). Format clearly using Markdown. Include necessary disclaimers.',
    backstory=(
        "You create clear, organized reports. You filter information based on relevance and present key details "
        "in a structured format. You ensure all necessary context and disclaimers are included."
    ),
    tools=[],
    llm=llm,
    allow_delegation=False,
    verbose=True,
    max_iter=3
)
print("- Agent 'report_compiler' defined.")

print("All grant finding agents (URL Input Version) defined.")

Defining grant finding agents (URL Input Version)...
- Agent 'project_info_extractor' (NEW) defined.
- Agent 'org_info_extractor' (NEW) defined.
- Agent 'opportunity_searcher' (Modified) defined.
- Agent 'grant_info_extractor' defined.
- Agent 'eligibility_matcher' (Modified) defined.
- Agent 'report_compiler' defined.
All grant finding agents (URL Input Version) defined.


In [13]:
# @title 6. Define tasks for grant finding crew

print("Defining grant finding tasks (URL Input Version)...")

# Verify agents are defined using the NEW variable names
if 'project_info_extractor' not in locals() or \
   'org_info_extractor' not in locals() or \
   'opportunity_searcher' not in locals() or \
   'grant_info_extractor' not in locals() or \
   'eligibility_matcher' not in locals() or \
   'report_compiler' not in locals():
    raise ValueError("One or more required agents (using updated names) are not defined.")

# --- Task 0a: Extract Project Info from URL ---
task_extract_project_info = Task(
    description=(
        f"Visit the project information URL: '{project_info_url}' using the web reading tool. "
        f"Extract and summarize the project's goals, activities, target location/beneficiaries, and any relevant keywords mentioned on the page. "
        f"If specific keywords were provided ('{project_keywords_override}'), ensure they are included or noted."
    ),
    agent=project_info_extractor,
    expected_output=(
        "A structured text summary containing the extracted project description, goals, location/beneficiaries, and relevant keywords."
    ),
    tools=[web_reader_tool] # Explicitly state tool needed
)
print("- Task 'task_extract_project_info' (NEW) defined.")

# --- Task 0b: Extract Organization Info from URL ---
task_extract_org_info = Task(
    description=(
        f"Visit the organization information URL: '{organization_info_url}' using the web reading tool. "
        f"Extract key details like organization type, primary location, mission/focus areas, and legal status (e.g., non-profit status) if available on the page. "
        f"Focus on details relevant for grant eligibility."
    ),
    agent=org_info_extractor,
    expected_output=(
        "A structured text summary containing the extracted organizational details (type, location, focus, status etc.)."
    ),
    tools=[web_reader_tool] # Explicitly state tool needed
)
print("- Task 'task_extract_org_info' (NEW) defined.")


# --- Task 1: Search for Grant Opportunities ---
task_search_grants = Task(
    description=(
        f"Based on the extracted project details (provided in context) and considering the optional keywords '{project_keywords_override}', "
        f"use web search tools to find 5-7 potential grant funding opportunities. Focus on relevant funding sources and geographic scope based on the extracted project info. "
        f"Provide a list of promising leads with URLs."
    ),
    agent=opportunity_searcher,
    expected_output=(
        "A list of 5-7 potential grant opportunities, including names and URLs if found."
        " Example: [{'name': 'Clean Water Initiative Grant', 'url': 'https://...'}, ...]"
    ),
    context=[task_extract_project_info] # Depends on extracted project info
)
print("- Task 'task_search_grants' (Modified Context) defined.")

# --- Task 2: Extract Grant Details ---
task_extract_details = Task(
    description=(
        "For each potential grant lead identified in the previous task (context provided), use web reading tools to visit the URL and extract the specified key information: "
        "Grant Name, Funding Agency, Deadline, Funding Range, Purpose Summary, and the **verbatim text of the Eligibility Criteria section**. "
        "Structure the extracted data clearly for each grant. If information is not found, note that."
    ),
    agent=grant_info_extractor,
    expected_output=(
        "A structured list, where each item represents a grant and contains the extracted details: "
        "{'grant_name': '...', 'agency': '...', 'deadline': '...', 'funding_range': '...', 'purpose': '...', 'eligibility_criteria_text': '...'}. "
        "Indicate if specific fields were not found."
    ),
    context=[task_search_grants], # Needs the list of leads
    tools=available_tools
)
print("- Task 'task_extract_details' defined.")

# --- Task 3: Perform Preliminary Eligibility Check ---
task_check_eligibility = Task(
    description=(
        "Take the structured grant details, including the 'eligibility_criteria_text' (from Task 2 context). "
        f"Compare the extracted eligibility criteria *strictly* against the extracted organization profile (from Task 0b context). "
        "For each grant, add a preliminary 'eligibility_assessment' field ('Potentially Eligible', 'Likely Ineligible', 'Unclear / Needs Verification') "
        "and an 'eligibility_reasoning' field if not 'Potentially Eligible'. **Emphasize this is preliminary.**"
    ),
    agent=eligibility_matcher,
    expected_output=(
        "The list of structured grant details with 'eligibility_assessment' and 'eligibility_reasoning' fields added to each item."
    ),
    # Depends on extracted grant details AND extracted org details
    context=[task_extract_details, task_extract_org_info]
)
print("- Task 'task_check_eligibility' (Modified Context) defined.")

# --- Task 4: Compile Final Report ---
task_compile_report = Task(
    description=(
        "Generate a final report summarizing the grant opportunities assessed (context provided from the eligibility check task). "
        "Filter to include primarily 'Potentially Eligible' or 'Unclear' opportunities. "
        "Present key details for each (Name, Agency, Deadline, Funding, Purpose, Full Eligibility Text, Preliminary Assessment/Reasoning) using Markdown. "
        "**Include prominent disclaimers at the start and end.**"
    ),
    agent=report_compiler,
    expected_output=(
        "A well-formatted Markdown report starting and ending with disclaimers. "
        "Lists potentially suitable grants with key details and preliminary eligibility check results."
    ),
    context=[task_check_eligibility] # Depends on the final annotated list
)
print("- Task 'task_compile_report' (Modified Context) defined.")

print("All grant finding tasks (URL Input Version) defined.")

Defining grant finding tasks (URL Input Version)...
- Task 'task_extract_project_info' (NEW) defined.
- Task 'task_extract_org_info' (NEW) defined.
- Task 'task_search_grants' (Modified Context) defined.
- Task 'task_extract_details' defined.
- Task 'task_check_eligibility' (Modified Context) defined.
- Task 'task_compile_report' (Modified Context) defined.
All grant finding tasks (URL Input Version) defined.


In [10]:
# @title 7. Create and run grant finding crew

print("Creating the grant finding crew (URL Input Version)...")

# Verify all components are ready using NEW agent/task names
if 'project_info_extractor' not in locals() or \
   'org_info_extractor' not in locals() or \
   'opportunity_searcher' not in locals() or \
   'grant_info_extractor' not in locals() or \
   'eligibility_matcher' not in locals() or \
   'report_compiler' not in locals() or \
   'task_extract_project_info' not in locals() or \
   'task_extract_org_info' not in locals() or \
   'task_search_grants' not in locals() or \
   'task_extract_details' not in locals() or \
   'task_check_eligibility' not in locals() or \
   'task_compile_report' not in locals():
    raise ValueError("Missing agents or tasks needed to create the grant finding crew.")

# Assemble the Crew with NEW agents and tasks in order
grant_finding_crew = Crew(
    agents=[
        project_info_extractor,   # New
        org_info_extractor,       # New
        opportunity_searcher,
        grant_info_extractor,
        eligibility_matcher,
        report_compiler
    ],
    tasks=[
        task_extract_project_info,  # New
        task_extract_org_info,    # New
        task_search_grants,
        task_extract_details,
        task_check_eligibility,
        task_compile_report
    ],
    process=Process.sequential,
    memory=True,
    cache=True,
    verbose=True
)

print("Grant finding crew created. Starting the process with kickoff()...")

# Define initial inputs - now passing URLs
crew_inputs = {
    'project_url': project_info_url,
    'organization_url': organization_info_url,
    'keywords_override': project_keywords_override
}

# Kick off the crew's work
result = grant_finding_crew.kickoff(inputs=crew_inputs)

print("\n\n***************************************")
print("   GRANT FINDING CREW FINISHED WORK!   ")
print("***************************************")

Creating the grant finding crew (URL Input Version)...


/usr/local/lib/python3.11/dist-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing this attribute on the instance is deprecated, and will be removed in Pydantic V3. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


Grant finding crew created. Starting the process with kickoff()...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 043be707-9434-4b4d-b250-a2b05e1f2259                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: d09bb2e7-b70d-4d0f-b399-1f89533b016a
       Status: Executing Task...

...

🚀 Crew: crew
├── 📋 Task: d09bb2e7-b70d-4d0f-b399-1f89533b016a
│      Assigned to: Web Content Analyst for Projects
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Content Analyst for Projects
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (1)
├── 📋 Task: 3bee880c-e797-47f7-b132-f528981a48fa
│      Assigned to: Web Content Analyst for Organizations
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Content Analyst for Organizations
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (2)
├── 📋 Task: edd3ded1-880e-4034-9ac6-1ad04d3a7765
│      Assigned to: Grant Prospecting Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Grant Prospecting Specialist
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (1)
├── 📋 Task: e40f6ac7-2518-4aa6-af6c-8f777379400e
│      Assigned to: Grant Information Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Grant Information Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (3)
├── 📋 Task: eb18672b-a586-4daf-aa60-91a93a35c269
│      Assigned to: Preliminary Eligibility Assessor
│      Status: ✅ Completed
│   └── 🤖 Agent: Preliminary Eligibility Assessor
│           Status: ✅ Completed
└── 📋 Task: 259b7566-5a4c-4ca1-a501-d9bc5fa8810c
       Status: Executing Task...
    └── 🤖 Agent: Funding Opportunity Report Generator
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: d09bb2e7-b70d-4d0f-b399-1f89533b016a
│      Assigned to: Web Content Analyst for Projects
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Content Analyst for Projects
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (1)
├── 📋 Task: 3bee880c-e797-47f7-b132-f528981a48fa
│      Assigned to: Web Content Analyst for Organizations
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Content Analyst for Organizations
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (2)
├── 📋 Task: edd3ded1-880e-4034-9ac6-1ad04d3a7765
│      Assigned to: Grant Prospecting Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Grant Prospecting Specialist
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (1)
├── 📋 Task: e40f6ac7-2518-4aa6-af6c-8f777379400e
│      Assigned to: Grant Information Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Grant Information Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search in a specific website (3)
├── 📋 Task: eb18672b-a586-4daf-aa60-91a93a35c269
│      Assigned to: Preliminary Eligibility Assessor
│      Status: ✅ Completed
│   └── 🤖 Agent: Preliminary Eligibility Assessor
│           Status: ✅ Completed
└── 📋 Task: 259b7566-5a4c-4ca1-a501-d9bc5fa8810c
       Assigned to: Funding Opportunity Report Generator
       Status: ✅ Completed
    └── 🤖 Agent: Funding Opportunity Report Generator
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 259b7566-5a4c-4ca1-a501-d9bc5fa8810c                                                                     │
│  Agent: Funding Opportunity Report Generator                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 043be707-9434-4b4d-b250-a2b05e1f2259                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯



***************************************
   GRANT FINDING CREW FINISHED WORK!   
***************************************


In [11]:
# @title 8. Display final grant opportunities report

print("\n\n############################################################")
# Updated title to clearly indicate it's the output of this specific crew
print("## AI-Assisted Grant Opportunity Scan & Preliminary Check:")
# Reminder about verification
print("## REMINDER: VERIFY ALL DETAILS & ELIGIBILITY INDEPENDENTLY")
print("############################################################\n")

# Check if the 'result' variable exists (meaning Cell 7 completed)
if 'result' in locals() and result is not None:
    # Print the final output string generated by the crew's last task
    print(result)
else:
    # Message if Cell 7 failed or didn't produce a result
    print("!!! The 'result' variable is not defined or is empty. Ensure Cell 7 executed successfully and produced output.")

# Add the final critical disclaimer again for maximum emphasis
print("\n\n"+"*"*70)
print("!!! IMPORTANT DISCLAIMER !!!")
print("The above report is generated by an AI assistant for informational purposes ONLY.")
print("It may contain errors, omissions, or misinterpretations. Grant opportunities")
print("and eligibility criteria can change and are often complex.")
print("This is only a PRELIMINARY scan based on available text.")
print("You MUST read the full official grant guidelines and verify all details and")
print("eligibility requirements directly with the funding agency.")
print("This is NOT financial or legal advice.")
print("*"*70)



############################################################
## AI-Assisted Grant Opportunity Scan & Preliminary Check:
## REMINDER: VERIFY ALL DETAILS & ELIGIBILITY INDEPENDENTLY
############################################################

# Funding Opportunity Report

**Disclaimer: This report provides a summary of potentially relevant grant opportunities based on preliminary information. It is crucial for interested parties to verify details directly from the funding agencies before proceeding with applications.**

---

### **Grant Opportunity: 10 Grants for NGOs Working in Conflict Zones**
- **Funding Agency:** Various agencies including OCHA, ECHO, USAID, ICRC, Ford Foundation, Open Society Foundations, Wellcome Trust, Global Fund for Women, Humanitarian Innovation Fund
- **Deadline:** Not specified
- **Funding Range:** Not specified
- **Purpose Summary:** 
  - To support NGOs delivering critical services in conflict-affected areas through various types of grants focusing on he